## Language model for error row finder

In [5]:
from __future__ import generator_stop
import numpy as np
import random
import nltk, re, string, collections
from nltk.util import ngrams
import pandas as pd
from nltk.probability import *
import string
import copy

li = ['timestamp', 'description','limit6','mailid','float']
temp = pd.read_csv("./dataset.csv")
temp = temp.fillna("")
tempn = pd.DataFrame()
for i in temp.columns:
    tempn[str(i)] = '*' + temp[str(i)].map(str) + '*'
npdf = tempn[tempn['output'] =="*0*"]


def map_parameter_init():
    d = {}
    pun = list(string.punctuation)
    num = list(string.digits)
    let = list(string.ascii_letters)
    for i in pun:
        d[i] = i
    for j in num:
        d[j] = 'n'
    for k in let:
        d[k] = 'c'
    d[' '] = ' '
    d['\n'] = '\n'
    return d

d = map_parameter_init()
vi = dict()
cfd = dict()
cols = temp.columns
rejli = {key:[] for key in cols}
rejrli ={key:[] for key in cols}
rejoli = []
rejpd = pd.DataFrame(columns=temp.columns, index = range(300)) #.reindex_like(temp)
mingram = dict()

In [6]:
def mapval(a):
    u,inv = np.unique(a,return_inverse = True)
    te = tuple(np.array([d[x] for x in u])[inv])
    lin.append(te)
    
for i in temp.columns:
    #distribution counting
    lin = []
    tokd = pd.DataFrame(npdf[str(i)].map(str).apply(list))
    val = tokd[str(i)].apply(mapval)
    vi[str(i)] = collections.Counter(lin)
    
    ##Language model implementation
    toke = list(npdf[str(i)].str.cat(sep='\n'))
    minv = npdf[str(i)].map(len).min()
    mingram[str(i)] = minv
    u,inv = np.unique(toke,return_inverse = True)
    te = np.array([d[x] for x in u])[inv].reshape(len(toke))
    total = len(te)
    bigr = nltk.ngrams(te,mingram[str(i)])
    condition_pairs = (((w[:-1]), w[-1]) for w in bigr)
    cfd[str(i)] = nltk.ConditionalFreqDist(condition_pairs)
print(cfd)

{'timestamp': <ConditionalFreqDist with 17 conditions>, 'description': <ConditionalFreqDist with 1569 conditions>, 'ipaddress': <ConditionalFreqDist with 159 conditions>, 'limit6': <ConditionalFreqDist with 336 conditions>, 'float': <ConditionalFreqDist with 28 conditions>, 'year': <ConditionalFreqDist with 7 conditions>, 'money': <ConditionalFreqDist with 15 conditions>, 'country': <ConditionalFreqDist with 17 conditions>, 'Enum': <ConditionalFreqDist with 5 conditions>, 'mailid': <ConditionalFreqDist with 860 conditions>, 'bool': <ConditionalFreqDist with 8 conditions>, 'longnumber': <ConditionalFreqDist with 11 conditions>, 'stockprice': <ConditionalFreqDist with 20 conditions>, 'output': <ConditionalFreqDist with 4 conditions>}


In [7]:
#run on actual data
def tmapval(a, va,i):
    u,inv = np.unique(a,return_inverse = True)
    te = tuple(np.array([d[x] for x in u])[inv])
    try:
        tel = vi[str(i)].get(te)/va
    except:
        rejli[str(i)].append(te)
        rejrli[str(i)].append(a)
        
def feature_mapping():
    count = 0
    for i in temp.columns:
        ind = 0
        tdf = pd.DataFrame(tempn[str(i)].map(str).apply(list))
        count = len(tdf.index)
        tempa = tdf[str(i)].apply(tmapval, args = (count,i))
        print("Number of rejected items in",str(i),"is", len(rejli[str(i)]),"\n")
        if rejli[str(i)] !=[]:
            for j in rejli[str(i)]:
                tr = nltk.ngrams(j,mingram[str(i)])
                if list(tr) == []:
                    vi = 0
                    tem = copy.deepcopy(rejrli[str(i)][rejli[str(i)].index(j)])
                    tem.pop(0)
                    tem.pop(-1)
                    rejpd.at[ind, str(i)] = str("".join(tem))
                    ind+=1
                    continue
                else:
                    tr = nltk.ngrams(j,mingram[str(i)])
                    cpairs = (((w[:-1]), w[-1]) for w in tr)
                    co = 0
                    vi = 0
                    for k in list(cpairs):
                        v = cfd[str(i)][k[0]][k[1]]
                        if co == 0:
                            vi = v/total
                            co = 1
                        else:
                            vi *= v/total
                    if vi ==0:
#                         print("non empty, zero prob so reject","".join(rejrli[rejli.index(j)]))
                        tel = copy.deepcopy(rejrli[str(i)][rejli[str(i)].index(j)])
                        tel.pop(0)
                        tel.pop(-1)
                        rejpd.at[ind, str(i)] = str("".join(tel))
                        ind+=1
                    elif vi!=0:
                        print("Accepted after language mode:","".join(rejrli[str(i)][rejli[str(i)].index(j)]))
                
x = feature_mapping()

Number of rejected items in timestamp is 17 

Number of rejected items in description is 261 

Accepted after language mode: *seize clicks-and-mortar e-business*
Accepted after language mode: *scale innovative niches*
Accepted after language mode: *syndicate vertical convergence*
Accepted after language mode: *incubate granular architectures*
Accepted after language mode: *disintermediate extensible markets*
Accepted after language mode: *engineer customized e-markets*
Accepted after language mode: *harness viral solutions*
Accepted after language mode: *exploit granular bandwidth*
Accepted after language mode: *redefine world-class convergence*
Accepted after language mode: *extend B2C niches*
Accepted after language mode: *brand revolutionary architectures*
Accepted after language mode: *cultivate magnetic ROI*
Accepted after language mode: *evolve turn-key mindshare*
Accepted after language mode: *engage user-centric interfaces*
Accepted after language mode: *brand extensible portal

C:\Users\gokul\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: generator 'ngrams' raised StopIteration



Accepted after language mode: *orchestrate revolutionary niches*
Accepted after language mode: *transition next-generation interfaces*
Accepted after language mode: *integrate back-end experiences*
Accepted after language mode: *monetize front-end metrics*
Accepted after language mode: *extend B2C niches*
Accepted after language mode: *harness interactive partnerships*
Accepted after language mode: *reinvent B2B systems*
Accepted after language mode: *expedite next-generation interfaces*
Accepted after language mode: *reinvent 24/7 communities*
Accepted after language mode: *transform one-to-one niches*
Accepted after language mode: *streamline robust mindshare*
Accepted after language mode: *incubate vertical action-items*
Accepted after language mode: *disintermediate bleeding-edge content*
Accepted after language mode: *reinvent mission-critical infomediaries*
Accepted after language mode: *incubate best-of-breed communities*
Accepted after language mode: *productize virtual e-tail

Number of rejected items in year is 10 

Number of rejected items in money is 25 

Number of rejected items in country is 25 

Accepted after language mode: *Cook Islands*
Accepted after language mode: *United Kingdom*
Accepted after language mode: *Costa Rica*
Accepted after language mode: *Sierra Leone*
Accepted after language mode: *Sierra Leone*
Number of rejected items in Enum is 24 

Number of rejected items in mailid is 150 

Accepted after language mode: *plaguerre6v@imdb.com*
Accepted after language mode: *acausnett8a@tumblr.com*
Accepted after language mode: *kmereweather68@addthis.com*
Accepted after language mode: *vgwinnel5v@topsy.com*
Accepted after language mode: *rcartmel1z@telegraph.co.uk*
Accepted after language mode: *ctapin13@webnode.com*
Accepted after language mode: *ttroy3v@comsenz.com*
Accepted after language mode: *backenheadah@nba.com*
Accepted after language mode: *gdrakeford5y@dion.ne.jp*
Accepted after language mode: *rcartmel1z@telegraph.co.uk*
Accepted af

In [4]:
rejpd

,timestamp,description,ipaddress,limit6,float,year,money,country,Enum,mailid,bool,longnumber,stockprice,output
0,,aggregate out-of-the-box web services,,*1491e1,#22.7064041,87563,"""3.85","""@rmenia",34%,@sdockerty4jmapquest.com,",.",,n/a,NaN
1,*7:17:52 AM,enhance B2B functionalities,242.ab.180.0,,,5,^,^&#,90.89,msolleym1@vk.com,,kkkd,n/a,NaN
2,,grow open-source e-business,0,*7311fe,,ab&^,#0.94,,!@ER,amcgeechan1r%vinaora.com,,89uy,n/a,NaN
3,6:3a:58,scale back-end infomediaries,7.111.180.,*(#&,,ab,,Czech Republic.,#,,.,kkkd,n/a,NaN
4,,grow plug-and-play niches,87.241.12&.184,a%^df,16.8,kckd,0)19,,$,bdayshb8@google.fr,6532343,,n/a,NaN
5,,transition B2B e-markets,49.1*3.90.241,,78dfj8.78,32,6.76B,,%^,hbetunr9@miibeian.gov.cn,#$%,%^dfklj,n/a,NaN
6,9:22: AM,utilize one-to-one ROI,164.7.7.224,7829,26.76c7964,-1234,4.31%,,&*,,gh!@,,n/a,NaN
7,%^fj,incubate viral e-markets,,,40*5753726,12ab,d,,10o,brycen5@1688.com,123,,n/a,NaN
8,ab12,evolve front-end relationships,!@#$%,abcd,$45.26,%$,df,C0lombia,2l1,,,"$5,100,140,000,000,000",n/a,NaN
9,83ejr,benchmark plug-and-play ROI,,,,275634,5*11,chin78,10o,,<>,$%3548130000000000,n/a,NaN


In [ ]:
# condition_pairs.throw()
# a = {}
# k = 0
# while k < 10:
#     <dynamically create key> 
#     key = ...
#     <calculate value> 
#     value = ...
#     a[key] = value 
#     k += 1

# t = nltk.ngrams("timestamp,description,ipaddress,limit6,float,year,money,country,Enum,mailid,bool,longnumber,stockprice,output", 3)
# # list(t)
# for i in list(t):
#     print(i)
#     print(i[:-1])
#     print(i[-1])
    
# t = nltk.ngrams("timestamp,description,ipalongnumber,stockprice,output", 3)
# if list(t) == []:
#     print("in")
# else:
#     print("out")

    
# # collections.Counter(vi.most_common(1))
# l = list(vi.keys())
# for i in l:
#     print(collections.Counter(i))
# npdf['mailid'].map(len).sum()/len(npdf['mailid'].index) $%average length of list in a columnx
# fdist = nltk.FreqDist(w.lower() for w in te)
# total = 0
# for word in fdist:
#     total += fdist[word]
    
# v = list('**nncn')
# for w in v:
#     if w in fdist:
#         print(w,'freq',fdist[w],'prob',fdist[w]/total)
#     else:
#         print(w,'freq',0,'prob',0)
# cfd.items()
# cfd.keys()
# cfd[v[0]][v[1]]

# toke = list(data)
# u,inv = np.unique(toke,return_inverse = True)
# te = np.array([d[x] for x in u])[inv].reshape(len(toke))
# esBigrams = ngrams(te, 3)
# # list(esBigrams)[:10]
# esBigramFreq = collections.Counter(esBigrams)
# esBigramFreq.most_common(20)
